# Лабораторная работа №3

## Подготовка входных данных

In [37]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [39]:
dataset = pd.read_csv('Australia Rainfall.csv', delimiter=',').sample(50000)

In [40]:
X = dataset.drop(columns=['Evaporation', 'Sunshine', 'Date', 'Cloud9am', 'Cloud3pm'])
X.dropna(inplace=True)

In [41]:
locations = X['Location'].unique()
X['Location'] = X['Location'].map({value: i for i, value in enumerate(locations)})

X['RainToday'] = X['RainToday'].map({'No': False, 'Yes': True})
X['RainTomorrow'] = X['RainTomorrow'].map({'No': False, 'Yes': True})

wgds = X['WindGustDir'].unique()
X['WindGustDir'] = X['WindGustDir'].map({value: i for i, value in enumerate(wgds)})

wd9s = X['WindDir9am'].unique()
X['WindDir9am'] = X['WindDir9am'].map({value: i for i, value in enumerate(wd9s)})

wd9s = X['WindDir3pm'].unique()
X['WindDir3pm'] = X['WindDir3pm'].map({value: i for i, value in enumerate(wd9s)})
X

,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
90863,0,11.7,23.8,0.0,0,39.0,0,0,22.0,19.0,37.0,30.0,1016.6,1013.0,18.5,23.6,False,False
121634,1,20.6,35.2,0.0,1,35.0,1,1,13.0,13.0,57.0,27.0,1018.4,1015.9,24.7,34.5,False,False
86656,2,19.1,29.8,0.0,2,33.0,2,2,7.0,13.0,47.0,48.0,1015.9,1013.5,26.2,27.1,False,False
74260,3,4.1,10.8,0.2,3,24.0,3,3,15.0,9.0,100.0,95.0,1021.9,1021.9,6.4,10.7,False,False
93051,0,22.3,30.8,0.0,4,52.0,4,4,26.0,41.0,69.0,74.0,1013.8,1008.0,26.8,25.3,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115330,42,19.4,38.0,0.0,3,52.0,7,10,26.0,31.0,23.0,13.0,1007.0,1002.0,27.9,37.0,False,False
118841,31,5.3,24.2,0.0,5,39.0,3,9,20.0,17.0,60.0,33.0,1029.9,1026.1,14.2,23.7,False,False
71824,10,11.1,28.9,0.0,11,41.0,5,15,20.0,22.0,62.0,31.0,1018.6,1016.7,14.5,26.7,False,False
81521,40,1.6,13.2,0.2,7,46.0,8,8,9.0,24.0,97.0,81.0,1020.4,1014.4,6.0,12.8,False,True


In [42]:
y = X['RainTomorrow']
X = X.drop('RainTomorrow', axis=1)

In [43]:
scaler = MinMaxScaler()
X_ans = scaler.fit_transform(X)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X_ans, y, test_size=0.3)

## Задание 1

Провести классификацию найденного датасета, методами линеной и логистической регрессий . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

In [53]:
def regression_to_classification(x):
    if x > 0.5:
        return 1
    else:
        return 0

### Линейная регрессия

In [56]:
from sklearn.linear_model import LinearRegression

In [58]:
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
predicted = pd.DataFrame(linear_model.predict(X_test),columns=['class'])
predicted = predicted['class'].apply(lambda x: regression_to_classification(x))
print('Evaluation:\n', metrics.classification_report(y_test, predicted,digits=5))

Evaluation:
               precision    recall  f1-score   support

       False    0.86089   0.95982   0.90767      9085
        True    0.76206   0.45345   0.56858      2578

    accuracy                        0.84790     11663
   macro avg    0.81148   0.70664   0.73813     11663
weighted avg    0.83905   0.84790   0.83272     11663



### Логическая регрессия

In [60]:
from sklearn.linear_model import LogisticRegression

In [61]:
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)
predicted = pd.DataFrame(logistic_model.predict(X_test),columns=['class'])
predicted = predicted['class'].apply(lambda x: regression_to_classification(x))
print('Evaluation:\n', metrics.classification_report(y_test, predicted,digits=5))

Evaluation:
               precision    recall  f1-score   support

       False    0.87048   0.94761   0.90740      9085
        True    0.73153   0.50310   0.59618      2578

    accuracy                        0.84935     11663
   macro avg    0.80100   0.72535   0.75179     11663
weighted avg    0.83976   0.84935   0.83861     11663

